In [1]:
import pandas as pd
import os 

raw_df = pd.DataFrame(os.listdir('./Cleaned/'), columns=['image_name'])

def convert_format(df):
    split_names = df.image_name.apply(lambda x: x.split('_')).apply(lambda x: [x[0], ' '.join(x[1:-1]), x[-1]])
    df['split_name'] = split_names
    df = pd.DataFrame(df['split_name'].to_list(), columns=['age', 'name_id', 'image_id'])
    return df

df = convert_format(raw_df)

In [2]:
df

,age,name_id,image_id
0,14,0,0001.jpg
1,14,0,0002.jpg
2,14,10,0008.jpg
3,14,10,0009.jpg
4,14,11,0002.jpg
...,...,...,...
163300,62,1999,0003.jpg
163301,62,1999,0004.jpg
163302,62,1999,0005.jpg
163303,62,1999,0006.jpg


In [3]:
import random
def generate_samples(count):
    sampled_data = []
    for i in range(count):
        name_id_groups = df.groupby('name_id')
        name_ids = df['name_id'].unique().tolist()
        pos_id = random.sample(name_ids, 1)[0]
        table = name_id_groups.get_group(pos_id)
        common_ages = table.age.unique()
        for common_age in common_ages:
            anchor = df[(df['age'] == common_age) & (df['name_id'] == pos_id)].sample().values.tolist()[0]
            positive = df[(df['age'] != common_age) & (df['name_id'] == pos_id)].sample().values.tolist()[0]
            negative = df[(df['age'] == common_age) & (df['name_id'] != pos_id)].sample().values.tolist()[0]
            sampled_data.append(['_'.join(anchor), '_'.join(positive), '_'.join(negative)])
    sampled_df = pd.DataFrame(sampled_data, columns=['anchor', 'positive', 'negative'])
    return sampled_df

In [4]:
# testing
generate_samples(1)

,anchor,positive,negative
0,21_358_0009.jpg,23_358_0015.jpg,21_42_0010.jpg
1,22_358_0015.jpg,27_358_0011.jpg,22_289_0015.jpg
2,23_358_0005.jpg,24_358_0004.jpg,23_495_0003.jpg
3,24_358_0001.jpg,22_358_0019.jpg,24_452_0006.jpg
4,25_358_0004.jpg,30_358_0012.jpg,25_228_0003.jpg
5,26_358_0001.jpg,23_358_0005.jpg,26_629_0009.jpg
6,27_358_0007.jpg,30_358_0004.jpg,27_402_0010.jpg
7,28_358_0001.jpg,25_358_0012.jpg,28_665_0016.jpg
8,29_358_0005.jpg,23_358_0010.jpg,29_701_0009.jpg
9,30_358_0007.jpg,25_358_0003.jpg,30_551_0009.jpg


##### Generate samples

In [5]:
training_samples = generate_samples(10000)

In [6]:
training_samples.to_csv('./training_data.csv')

In [8]:
test_samples = generate_samples(5000)
test_samples.to_csv('./test_data.csv')